In [5]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import CLIPModel, CLIPProcessor, AdamW, get_scheduler
from tqdm import tqdm

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [2]:
# train dataset
data_dir = "/Users/anjonghyeon/Desktop/KU/3-2/DeepLearning/20242R0136COSE47402/FinalProject/data/train"
class_candidate = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]
text_inputs = []
for folder in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, folder)):
        folder = folder.replace('_', ' ')
        text_inputs.append(f"a photo of {folder}")

In [3]:
image_paths = []
image_labels = []

for class_name in class_candidate:
    class_folder = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_name)
        image_paths.append(img_path)
        class_name = class_name.replace('_', ' ')
        image_labels.append(f"a photo of {class_name}")

print(f"train dataset size : {len(image_paths)}")

train dataset size : 75750


In [9]:
class Food101DataSet(Dataset):
    def __init__(self, image_paths, image_labels, processor):
        self.image_paths = image_paths
        self.image_labels = image_labels
        self.processor = processor
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.image_labels[idx]
        input = self.processor(text=label, images=image, return_tensors='pt', padding=True)
        input = {k: v.squeeze(0) for k, v in input.items()}
        return input

train_dataset = Food101DataSet(image_paths, image_labels, processor)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

In [11]:
optimizer = AdamW(model.parameters(), lr=5e-6)
epochs = 8
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

/Users/anjonghyeon/Desktop/KU/3-2/DeepLearning/20242R0136COSE47402/FinalProject/dl3_2env/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
model.train()
losses = []

for epoch in range(epochs):
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1} / {epochs}")
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        progress_bar.set_postfix(loss=loss.item())
print("Train completed.")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.show()

In [16]:
# test dataset
data_dir = "/Users/anjonghyeon/Desktop/KU/3-2/DeepLearning/20242R0136COSE47402/FinalProject/data/test"
class_candidate = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]
text_inputs = []
for folder in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, folder)):
        folder = folder.replace('_', ' ')
        text_inputs.append(f"a photo of {folder}")

In [17]:
image_paths = []
image_labels = []

for class_name in class_candidate:
    class_folder = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_name)
        image_paths.append(img_path)
        class_name = class_name.replace('_', ' ')
        image_labels.append(f"a photo of {class_name}")

print(f"test dataset size : {len(image_paths)}")

test dataset size : 25250


In [ ]:
ans = 0
model.eval()

for idx, img_path in enumerate(image_paths):
    image = Image.open(img_path).convert('RGB')
    inputs = processor(text=text_inputs, images=image, return_tensors='pt', padding=True)
    outputs = model(**inputs)

    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    if text_inputs[torch.argmax(probs).item()] == image_labels[idx]:
        ans += 1
    if idx % 100 == 0:
        print(f"# {idx} finished.")

accuracy = ans / len(image_paths) * 100
accuracy